In [29]:
from typing import List, Dict
from hlathena.peptide_dataset_train import PeptideDatasetTrain
from hlathena import peptide_nn
from hlathena.training_evaluation import TrainingEvaluation

import torch.utils.data as torch_data
import torch
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import os
import datetime
import random


def create_config_dict(device, epochs, lr, dr, batch_size, decoy_mul, eval_decoy_ratio, fold, aa_features, featname,
                       pepfeats_dict, seed):
    return {
        'device': str(device),
        'epochs': epochs,
        'learning_rate': lr,
        'batch_size': batch_size,
        'dropout_rate': dr,
        'decoy_mul': decoy_mul,
        'eval_decoy_ratio': eval_decoy_ratio,
        'fold #': fold,
        'aa_feature_files': aa_features,
        'feature_set_name': featname,
        'feature_set_cols': pepfeats_dict[featname],
        'all_feature_sets': pepfeats_dict,
        'seed': seed
    }


def make_subdir(output_dir, subdir):
    subdir_path = os.path.join(output_dir, subdir)
    os.makedirs(subdir_path)
    return subdir_path


def make_output_dirs(output_dir, out_path):
    dir_name = '_'.join([get_currtime(), out_path])
    model_path = os.path.join(output_dir, dir_name)
    os.makedirs(model_path)

    models_dir = make_subdir(model_path, 'models')
    configs_dir = make_subdir(model_path, 'configs')
    eval_dir = make_subdir(model_path, 'eval')
    return models_dir, configs_dir, eval_dir


def get_currtime():
    return str(datetime.datetime.now()).replace(" ", "_")



In [2]:
decoy_mul = 1
decoy_ratio = 1
# for fold in range(folds):
fold=0
feat_set = []
seed=1
folds=5
batch_size = 5000
epochs = 100
learning_rate = 0.01
pred_replicates = 10
dropout_rate = 0.1 #0.0
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [3]:
test_file = "/Users/cleoforman/PycharmProjects/hlathenav2/notebooks/input/test_training.csv"
mini_df = pd.read_csv(test_file, index_col=0)

In [4]:
mini_df

,mhc,pep,tgt,src,len,tgt_weight
0,HLA-A*01:01,KSSFLSSPE,0,bigmhc,9,0.3
1,HLA-A*01:01,RTEAAFSYY,1,bigmhc,9,1.0
2,HLA-A*01:01,ASPQTLVLY,1,this,9,1.0
3,HLA-A*01:01,GVMLDDYIR,0,bigmhc,9,0.3
5,HLA-A*01:01,CRVDAGPVL,0,bigmhc,9,0.3
...,...,...,...,...,...,...
1495,HLA-C*16:01,FLKVSPELK,0,bigmhc,9,0.3
1496,HLA-C*16:01,VGLVLRYGI,0,bigmhc,9,0.3
1497,HLA-C*16:01,NATAFFRQH,1,bigmhc,9,1.0
1498,HLA-C*16:01,FSIVRPRRL,1,this,9,1.0


In [5]:

peptide_dataset = PeptideDatasetTrain(mini_df, pep_col_name='pep', allele_col_name='mhc', target_col_name='tgt', folds=5)
feature_dims = peptide_dataset.feature_dimensions()
# peptide_dataset.set_feat_cols(['len','tgt_weight'])
# peptide_dataset.set_feat_cols(['len'])
# peptide_dataset.set_feat_cols([])

In [59]:
# type(peptide_dataset.pep_df.iloc[0][peptide_dataset.peptide_feature_cols].values)

numpy.ndarray

In [67]:
# peptide_dataset.pep_df[peptide_dataset.peptide_feature_cols].iloc[0].values
# len(peptide_dataset.pep_df.iloc[0][peptide_dataset.peptide_feature_cols].values)


0

In [ ]:
import argparse
import random
import glob
import os
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt


from trainer2 import trainer
from classes.training_evaluation import TrainingEvaluation
from classes.peptide_dataset import PeptideDataset

def get_aa_feature_files_from_dir(aa_dir):
    # setting the path for joining multiple files
    if (not os.path.isdir(aa_dir)):
        return []
    elif not len(os.listdir(aa_dir)):
        print("Amino acid feature directory empty.")
        return [] 

    featurefiles = os.path.join(aa_dir, "*.txt")

    # list of merged files returned
    return glob.glob(featurefiles)

def parse_feature_sets(set_file):
    with open(set_file,'r') as f:
        featuresets = f.read().splitlines()
        featureset_dict = {}
        for s in featuresets:
            name, cols = s.split(":")
            cols = cols.split(",") if cols else []
            featureset_dict[name] = cols
    return featureset_dict

def get_dedup_pep_df(df):
    pep_grp = df.groupby("features")
    results=[]
    index=[]
    for p, d in pep_grp:
        r = [p] + list(d.mean(axis=0,numeric_only=True)) + list(d.std(axis=0,numeric_only=True))
        results.append(r)
        index.append(p)
    return pd.DataFrame(results, columns=["features", "mean_auc", "mean_prauc", "mean_ppv", "std_auc", "std_prauc", "std_ppv"],index=index)


def main():
    parser=argparse.ArgumentParser()
    parser.add_argument("--train_file", help="Peptide HLA train data file", required=True, type=str, action='store')
    parser.add_argument("--val_file", help="Peptide HLA validation data file", required=False, type=str, default=None, action='store')
    parser.add_argument("--pep_col", help="specify peptide sequence column name for hits and decoys", type=str, default='pep', action='store')
    parser.add_argument("--allele_col", help="Allele column name", type=str, default="", action='store')
    parser.add_argument("--target_col", help="Target column name", type=str, action='store')
    parser.add_argument("--fold_col", help="Fold column name", type=str, default=None, action='store')
    parser.add_argument("-kf", "--number_folds", help="specify number of cross folds", type=int, default=5, action='store')
    parser.add_argument("-e", "--epochs", type=int, default=4000, action="store")
    parser.add_argument("-lr", "--learning_rate", type=float, default=0.001, action="store")
    parser.add_argument("-dr", "--dropout_rate", type=float, default=0.1, action="store") #TODO: add default dr
    parser.add_argument("-b", "--batch_size", type=int, default=32, action="store")
    parser.add_argument("-pr", "--pred_replicates", type=int, default=100, action='store')
    parser.add_argument("-dm", "--decoy_mul", help="training ratio of hits to decoys e.g. 2.0 means 1:2 ratio of hits:decoys", type=int, default=1, action='store')
    parser.add_argument("--decoy_ratio", help="testing ratio of hits to decoys e.g. 2.0 means 1:2 ratio of hits:decoys", type=int, default=1, action='store')
    parser.add_argument("-rh", "--resample_hits", type=bool, default=0, action='store')
    parser.add_argument("--assign_folds", type=bool, default=True)
    parser.add_argument("--aa_feature_folder", help="comma-separated list of amino acid feature matrix files", type=str, default="", action='store')
    parser.add_argument("--feat_cols", help="peptide-level feature columns to train on e.g. 'exp;clev'", type=str, default="", action='store')
    parser.add_argument("--feat_sets", help="path to file containing feature combinations to run, one per row e.g. NAME:feat1,feat2", type=str, default="", action='store')
    parser.add_argument("--repetitions", help="number of times to repeat training", type=int, default=1, action='store')
    parser.add_argument("--seeds", help="optionally provide seeds for the training reps as comma-delimited list", type=str, default=None, action='store')
    parser.add_argument("-o", "--outdir", help="where to store", type=str, default="", action='store')
    parser.add_argument("-r", "--run_name", help="used to name output sub directory", type=str, default="", action='store')

    
    args = parser.parse_args()
    train_file = args.train_file
    val_file = args.val_file
    pep_col = args.pep_column
    allele_col = args.allele_col
    tgt_col = args.target_col
    fold_col = args.fold_col
    folds = args.number_folds
    epochs = args.epochs
    learning_rate = args.learning_rate
    dropout_rate = args.dropout_rate
    batch_size = args.batch_size
    pred_replicates = args.pred_replicates
    reps = args.repetitions
    decoy_mul = args.decoy_mul
    decoy_ratio = args.decoy_ratio
    resampling_hits = args.resample_hits
    aa_feature_folder = args.aa_feature_folder
    outdir = args.outdir
    assign_folds = args.assign_folds
    run_name = args.run_name
    seeds = args.seeds
    pep_feature_cols = [] if len(args.feat_cols) == 0 else args.feat_cols.split(";")
    pep_feature_sets_dict = {str(pep_feature_cols): [col for col in pep_feature_cols]} if not args.feat_sets \
                                                                                       else parse_feature_sets(args.feat_sets)
    # override peptide column input if feature set dict is provided
    pep_feature_cols = list({x for f in list(pep_feature_sets_dict.values()) if f for x in f}) if args.feat_sets \
                                                                                         else pep_feature_cols                                                                       
    pep_feature_cols.append(pep_col)
    
    seeds = None if seeds is None else seeds.split(';')

    # retrieve list of amino acid feature files
    aa_feature_files = get_aa_feature_files_from_dir(aa_feature_folder)
    
    train_df = pd.read_csv(train_file)

    assert(all([np.issubdtype(pep_df[c], np.number) for c in pep_df.columns if c!=pep_col]))
    
    # decoys_resampled = decoys_df.sample(N, random_state=1)

    rep_metrics = []
    
    peptide_dataset = PeptideDatasetTrain(train_df, 
                                          pep_col_name=pep_col, 
                                          allele_col_name=allele_col, 
                                          target_col_name=tgt_col, 
                                          fold_col_name=fold_col, 
                                          folds=folds, 
                                          aa_feature_files=aa_feature_files)
    
    if val_file is not None:
        val_df = pd.read_csv(val_file)
        val_dataset = PeptideDatasetTrain(val_df, 
                                          pep_col_name=pep_col, 
                                          allele_col_name=allele_col, 
                                          target_col_name=tgt_col, 
                                          fold_col_name=fold_col, 
                                          folds=folds, 
                                          aa_feature_files=aa_feature_files)
    else:
        val_dataset = None
        
    
    for rep in range(1,reps+1):
        print("\nTraining rep {} of {} reps...".format(rep,reps))

        rep_outdir = ''.join([outdir,'/rep',str(rep)])
        seed = random.randrange(0,100) if seeds is None else int(seeds[rep-1])
        print(f"SEED: {seed}\n")
        
        metrics = trainer(peptide_dataset=peptide_dataset, 
                          folds=folds, 
                          epochs=epochs, 
                          learning_rate=learning_rate, 
                          dropout_rate=dropout_rate, 
                          batch_size=batch_size, 
                          pred_replicates=pred_replicates,
                          decoy_mul=decoy_mul, 
                          decoy_ratio=decoy_ratio, 
                          resampling_hits=resampling_hits, 
                          aa_feature_files=aa_feature_files, 
                          output_dir=outdir, 
                          featsets_dict=pep_feature_sets_dict,
                          run_name=run_name,
                          seed=seed,
                          reassign_folds=assign_folds,
                          val_dataset=val_dataset)
        
        rep_metrics.extend(metrics)

        print("Training finished for rep {}. Outputs stored in {}".format(rep,rep_outdir))
    
    all_metrics = pd.DataFrame(rep_metrics, columns=["features","auc","prauc","ppv"])
    all_metrics.to_csv(os.path.join(outdir,'all_metrics.tsv'),sep='\t',index=False)
    
    # create summary plots if running multiple reps
    if reps>1:
        
        summ_metrics = get_dedup_pep_df(all_metrics)
        summ_metrics.to_csv(os.path.join(outdir,'summary_metrics.tsv'),sep='\t',index=False)

        TrainingEvaluation.save_feature_comparison_plots(summ_metrics, "mean_prauc", "std_prauc", outdir, "prauc_barplot.png")
        TrainingEvaluation.save_feature_comparison_plots(summ_metrics, "mean_ppv", "std_ppv", outdir, "ppv_barplot.png")

if __name__=="__main__":
    main()        
        

In [36]:
def trainer(peptide_dataset: PeptideDatasetTrain,
            folds: int,
            epochs: int,
            learning_rate: float,
            dropout_rate: float,
            batch_size: int,
            pred_replicates: int,
            decoy_mul: int,
            decoy_ratio: int,
            resampling_hits: bool,
            aa_feature_files: List[os.PathLike],
            output_dir: os.PathLike,
            featsets_dict: Dict[str, List[str]] = {},
            run_name: str = "", 
            seed: int = None,
            reassign_folds: bool = True,
            val_dataset: PeptideDatasetTrain = None):

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    metric_rows = []

    # peptide_dataset = PeptideDataset(hits_df, decoys_df, aa_feature_files)
    if reassign_folds:
        peptide_dataset.reassign_folds(folds, seed=seed)

    for featname in featsets_dict:
        print("Training feature set {} of {}...\n".format(str(featname), str(list(featsets_dict.keys()))))

        feat_set = featsets_dict[featname]
        run_out_path = '_'.join([run_name, featname]) if run_name else featname

        models_dir, configs_dir, eval_dir = make_output_dirs(output_dir=output_dir, out_path=run_out_path)

        eval_dict = {"peptide": [], "target": [], "pred_mean": [], "pred_var": [], "fold": []}
        cross_fold_dict = {"fold": [], "ppv": [], "prauc": []}

        # peptide_dataset = PeptideDataset(hits_df, decoys_df, aa_feature_files, feat_cols=feat_set, folds=folds)
        peptide_dataset.set_feat_cols(feat_set)
        feature_dims = peptide_dataset.feature_dimensions()

        # hits_df = hits_df.sample(hits_df.shape[0]*decoy_mul, replace=True, random_state=seed, ignore_index=True)
        for fold in range(folds):

            print("Training fold {} of {}...".format(str(fold + 1), folds))
            # resampling_hits = False
            train_ids = peptide_dataset.get_train_idxs(fold=fold, decoy_mul=decoy_mul, resampling_hits=resampling_hits,
                                                       seed=seed)
            test_ids = peptide_dataset.get_test_idxs(fold=fold, decoy_ratio=decoy_ratio, seed=seed)
            print(f"decoy mul: {decoy_mul}")
            print(f"decoy ratio: {decoy_ratio}")

            # print(f"Train split:")
            # unique, counts = np.unique(peptide_dataset.binds[train_ids], return_counts=True)
            # print(np.asarray((unique, counts)))
            # if len(feat_set):
            #     print(peptide_dataset.pep_df.iloc[train_ids].groupby('binds')[feat_set].describe())
            # print()
            # 
            # print(f"Test split:")
            # unique, counts = np.unique(peptide_dataset.binds[test_ids], return_counts=True)
            # print(np.asarray((unique, counts)))
            # if len(feat_set):
            #     print(peptide_dataset.pep_df.iloc[test_ids].groupby('binds')[feat_set].describe())
            # print()

            # Sample elements randomly from a given list of ids, no replacement.
            train_subsampler = peptide_nn.PeptideRandomSampler(train_ids, seed)
            test_subsampler = peptide_nn.PeptideRandomSampler(test_ids, seed)
            # return peptide_dataset, train_subsampler, test_subsampler, train_ids, test_ids
            # Define data loaders for training and testing data in this fold
            trainloader = torch_data.DataLoader(peptide_dataset, batch_size=batch_size, sampler=train_subsampler)
            testloader = torch_data.DataLoader(peptide_dataset, batch_size=batch_size, sampler=test_subsampler)


            try:
                val_subsampler = peptide_nn.PeptideRandomSampler([i for i in range(len(val_dataset))], seed)
                valloader = torch_data.DataLoader(val_dataset, batch_size=batch_size, shuffle=val_subsampler)
            except:
                valloader = None
            
            model = peptide_nn.PeptideNN2(feature_dims, dropout_rate)
            if torch.cuda.device_count() > 1:
                print("Let's use", torch.cuda.device_count(), "GPUs!")
                model = nn.DataParallel(model)
            model.to(device)
            optimizer_dict = peptide_nn.train(model, trainloader, learning_rate, epochs, device, valloader)
            
            # Create model and train
            model_config = create_config_dict(device=device, epochs=epochs, lr=learning_rate, dr=dropout_rate,
                batch_size=batch_size, decoy_mul=decoy_mul, eval_decoy_ratio=decoy_ratio,
                fold=fold, aa_features=aa_feature_files,
                featname=featname, pepfeats_dict=featsets_dict, seed=seed)

            # model = peptide_nn.PeptideNN(feature_dims, dropout_rate).to(device)
            # optimizer_dict = peptide_nn.train(model, trainloader, learning_rate, epochs, device)

            peptide_nn.save_model(model, fold, models_dir, configs_dir, optimizer_dict, model_config)
            
            inputs, targets, indices, preds = peptide_nn.evaluate(model, testloader, pred_replicates, device)
            inputs = torch.vstack(inputs).cpu()
            targets = [t.item() for t in torch.hstack(targets).cpu()]
            indices = [i.item() for i in torch.hstack(indices).cpu()]
            preds = torch.vstack(preds).cpu()
            
            input_peps, input_hla = [peptide_dataset.pep_at(i) for i in indices], [peptide_dataset.allele_at(i) for i in indices]
            pred_means = preds.mean(dim=-1).numpy()
            pred_vars = preds.var(dim=-1).numpy()


            for p, t, m, v in zip(input_peps, targets, pred_means, pred_vars):
                eval_dict['peptide'].append(p)
                eval_dict['target'].append(t)
                eval_dict['pred_mean'].append(m)
                eval_dict['pred_var'].append(v)
                eval_dict['fold'].append(str(fold + 1))

        eval_df = pd.DataFrame.from_dict(eval_dict)

        ## getting cross-fold metrics
        rows = []
        for fold, df in eval_df.groupby("fold"):
            train_eval = TrainingEvaluation(eval_df=df, decoy_ratio=True)
            cross_fold_dict['fold'].append(fold)
            cross_fold_dict['ppv'].append(train_eval.get_ppv())
            cross_fold_dict['prauc'].append(train_eval.get_prauc())

        crossfold_df = pd.DataFrame.from_dict(cross_fold_dict)
        crossfold_df.to_csv(os.path.join(eval_dir, "crossfold_eval.tsv"), index=False, sep='\t')

        eval_path_name = '{}_{}_eval.txt'.format(run_out_path, get_currtime())
        eval_df.to_csv(os.path.join(eval_dir, eval_path_name), index=False, sep='\t')

        train_eval = TrainingEvaluation(eval_df=eval_df, seed=seed, decoy_ratio=True)
        auc, prauc, ppv = train_eval.get_auc(), train_eval.get_prauc(), train_eval.get_ppv()

        print('\nTraining and evaluation finished for {run}.\n \
                 AUC: {auc}\n \
                 PPV: {ppv}\n \
                 PRAUC: {prauc}\n \
            '.format(run=featname, auc=auc, ppv=ppv, prauc=prauc))

        metric_rows.append([featname, auc, prauc, ppv])

    eval_metrics = pd.DataFrame(metric_rows, columns=['features', 'auc', 'prauc', 'ppv'])
    eval_metrics.to_csv(os.path.join(output_dir, 'eval_metrics.tsv'), sep='\t', index=False)
    return metric_rows



In [37]:
# def trainer(peptide_dataset: PeptideDatasetTrain,
#             folds: int,
#             epochs: int,
#             learning_rate: float,
#             dropout_rate: float,
#             batch_size: int,
#             pred_replicates: int,
#             decoy_mul: int,
#             decoy_ratio: int,
#             resampling_hits: bool,
#             aa_feature_files: List[os.PathLike],
#             output_dir: os.PathLike,
#             featsets_dict: Dict[str, List[str]] = {},
#             run_name: str = "", seed: int = None,
#             reassign_folds: bool = True):
data_dir = '/Users/cleoforman/PycharmProjects/hlathenav2/notebooks/input/'

aa_feature_files = [os.path.join(data_dir, 'aafeatmat_KideraFactors.txt')]

peptide_dataset = PeptideDatasetTrain(mini_df, pep_col_name='pep', allele_col_name='mhc', target_col_name='tgt', folds=5, aa_feature_files=aa_feature_files)

metrics = trainer(peptide_dataset=peptide_dataset, folds=folds, epochs=1, learning_rate=learning_rate, dropout_rate=dropout_rate, batch_size=batch_size, pred_replicates=pred_replicates,
        decoy_mul=decoy_mul, decoy_ratio=decoy_ratio, resampling_hits=False, aa_feature_files=aa_feature_files, output_dir='', featsets_dict={'P':[], 'PC': ['tgt_weight']},
                  val_dataset=peptide_dataset)

Training feature set P of ['P', 'PC']...

Training fold 1 of 5...
Resampling hits: False
decoy mul: 1
decoy ratio: 1
	In Model: input size torch.Size([750, 475]) output size torch.Size([750, 1])
Train outside: input size torch.Size([750, 475]) output_size torch.Size([750, 1])

Avg. train epoch 0 loss: 0.6942673921585083
	In Model: input size torch.Size([1450, 475]) output size torch.Size([1450, 1])
Eval outside: input size torch.Size([1450, 475]) output_size torch.Size([1450, 1])
Avg. validation epoch 0 loss: 0.579961359500885
	In Model: input size torch.Size([252, 475]) output size torch.Size([252, 1])
	In Model: input size torch.Size([252, 475]) output size torch.Size([252, 1])
	In Model: input size torch.Size([252, 475]) output size torch.Size([252, 1])
	In Model: input size torch.Size([252, 475]) output size torch.Size([252, 1])
	In Model: input size torch.Size([252, 475]) output size torch.Size([252, 1])
	In Model: input size torch.Size([252, 475]) output size torch.Size([252, 1])

KeyboardInterrupt: 

In [20]:
peptide_dataset.feature_dimensions()

476

In [32]:
metrics

[['P', 0.7389055581451867, 0.0059969419286015554, 0.001996007984031936],
 ['PC', 0.8194172672618834, 0.09823998676359039, 0.09181636726546906]]